In [1]:
# to run the code on google colab, uncomment the following lines
!git clone https://github.com/reddybhargava/data_analytics_project-1.git
%cd /content/data_analytics_project-1

Cloning into 'data_analytics_project-1'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 131 (delta 53), reused 101 (delta 33), pack-reused 0
Receiving objects: 100% (131/131), 14.99 MiB | 3.85 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/content/data_analytics_project-1


In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from functions import print_confusion_matrix
from functions import xgboost_classifier 
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [0]:
path = '/content/data_analytics_project-1/dataset_modified/'

train_values = pd.read_csv(path + 'train_values.csv')
train_labels = pd.read_csv(path + 'train_labels.csv')
test_values = pd.read_csv(path + 'test_values.csv')

In [0]:
# Train and test split
X_train, X_test, y_train, y_test = train_test_split(
                                        train_values, train_labels['damage_grade'], 
                                        test_size=0.3, random_state=10
                                    )
X_train = X_train.drop(columns = 'building_id')
X_test = X_test.drop (columns = 'building_id')

In [5]:
model, y_pred = xgboost_classifier(X_train, y_train, X_test, y_test)

Accuracy of XGBoostClassifier: 0.7444775584860772


## Log transformation

In [9]:
lg_X_train = X_train
lg_X_test = X_test

lg_X_train['age'] = np.log(lg_X_train['age'])
lg_X_test['age'] = np.log(lg_X_test['age'])

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
model, y_pred = xgboost_classifier(lg_X_train, y_train, lg_X_test, y_test)

Accuracy of XGBoostClassifier: 0.7437740627518195


## Without dropping building_id

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
                                        train_values, train_labels['damage_grade'], 
                                        test_size=0.3, random_state=10
                                    )

In [12]:
model, y_pred = xgboost_classifier(X_train, y_train, X_test, y_test)

Accuracy of XGBoostClassifier: 0.7392972717156342


## Normalizing the numeric columns

In [0]:
scalar = StandardScaler()
norm_X_train = scalar.fit_transform(X_train)
norm_X_test = scalar.transform(X_test)

In [19]:
model, y_pred = xgboost_classifier(norm_X_train, y_train, norm_X_test, y_test)

Accuracy of XGBoostClassifier: 0.7397577416507847


## One Hot Encoding

In [0]:
path = '/content/data_analytics_project-1/dataset/'
train_values = pd.read_csv(path + 'train_values.csv')
train_labels = pd.read_csv(path + 'train_labels.csv')

In [0]:
train_values_new = pd.get_dummies(train_values)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
                                        train_values_new, train_labels['damage_grade'], 
                                        test_size=0.3, random_state=10
                                    )

In [25]:
model, y_pred = xgboost_classifier(X_train, y_train, X_test, y_test)

Accuracy of XGBoostClassifier: 0.7398216958084445
